<a href="https://colab.research.google.com/github/gtbnhyujmj/-Reference-BingX_Market-Data/blob/main/Mark_Price_Kline_Candlestick_Data_%5BFormal%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mark Price Kline/Candlestick Data
# 標記價格 K 線/蠟燭圖資料

In [ ]:
# 取得最新標記價格 K 線資料

In [ ]:
# GET /openApi/swap/v1/market/markPriceKlines
# 不需要 API 金鑰簽名

In [ ]:
import time
import requests
import hmac
from hashlib import sha256

In [ ]:
APIURL = "https://open-api.bingx.com"  # API 基本網址
APIKEY = ""  # 使用者 API 金鑰，需填入有效金鑰
SECRETKEY = ""  # 使用者秘密金鑰，需填入有效密鑰

In [ ]:
def demo():
    payload = {}  # GET 請求通常無請求體
    path = '/openApi/swap/v1/market/markPriceKlines'  # API 路徑：取得標記價格 K 線資料
    method = "GET"  # HTTP 請求方法

    paramsMap = {
        "symbol": "KNC-USDT",  # 交易對，需包含連字號，如 KNC-USDT
        "interval": "1h",  # K 線時間間隔，例：1 小時
        "limit": "1000",  # 限制回傳資料筆數，例：1000 筆
        "startTime": "1702717199998"  # 起始時間戳，單位：毫秒
    }

    paramsStr = parseParam(paramsMap)  # 將參數字典轉成 URL 查詢字串並加上 timestamp
    return send_request(method, path, paramsStr, payload)  # 發送請求並回傳結果

In [ ]:
def get_sign(api_secret, payload):

    # 使用 HMAC-SHA256 演算法對參數字串產生簽名
    signature = hmac.new(api_secret.encode("utf-8"), payload.encode("utf-8"), digestmod=sha256).hexdigest()
    print("sign=" + signature)  # 輸出簽名以供調試
    return signature  # 回傳簽名

In [ ]:
def send_request(method, path, urlpa, payload):

    # 組合完整 URL，包含簽名參數
    url = "%s%s?%s&signature=%s" % (APIURL, path, urlpa, get_sign(SECRETKEY, urlpa))
    print(url)  # 輸出完整請求 URL 方便除錯

    headers = {
        'X-BX-APIKEY': APIKEY,  # HTTP 標頭帶入 API 金鑰
    }

    # 發送 HTTP 請求，帶入方法、URL、標頭與請求體
    response = requests.request(method, url, headers=headers, data=payload)
    return response.text  # 回傳 API 回應內容

In [ ]:
def parseParam(paramsMap):
    sortedKeys = sorted(paramsMap)  # 參數鍵排序，確保簽名一致

    # 將參數組成 URL 查詢字串，如 key=value&key2=value2
    paramsStr = "&".join(["%s=%s" % (x, paramsMap[x]) for x in sortedKeys])

    if paramsStr != "":
        # 若有其他參數，加上 timestamp 時間戳（毫秒）
        return paramsStr+"&timestamp="+str(int(time.time() * 1000))
    else:
        # 無其他參數時，只有 timestamp
        return paramsStr+"timestamp="+str(int(time.time() * 1000))

In [ ]:
if __name__ == '__main__':
    print("demo:", demo())  # 執行範例函式並印出結果